# 04 实战项目-基础爬虫

内容导航：

1. 基础爬虫架构及运行流程
2. URL管理器
3. HTML下载器
4. HTML解析器
5. 数据存储器
6. 爬虫调度器

## 4.1 基础爬虫架构及运行流程

### 爬虫架构

基础爬虫框架主要包括五大模块，分别为**爬虫调度器**、**URL管理器**、**HTML下载器**、**HTML解析器**和**数据存储器**。如下图所示：

![基础爬虫架构](images/spider-architecture.jpeg)

### 爬虫框架各模块的功能

爬虫框架五大模块的功能如下：

* 爬虫调度器主要负责协调其他四个模块的工作
* URL管理器负责管理URL链接，维护已经爬取的URL集合和未爬取的URL集合，提供获取新URL的接口
* HTML下载器用于从URL管理器中获取未爬取的URL链接并下载HTML网页
* HTML解析器用于从HTML下载器中获取已经下载的HTML网页，并从中解析出新的URL链接交给URL管理器，解析出目标数据则交给数据存储器
* 数据存储器用于将HTML解析器解析出来的目标数据通过文件或者数据库的形式存储起来。

### 爬虫框架的运行流程

爬虫框架的动态运行流程如下图所示：

![运行流程](images/spider-flow.jpeg)

## 4.2 URL管理器

主要包括:

* 已爬取的URL集合: used_urls: set
* 未爬取的URL集合: new_urls: set

主要功能：URL**去重**

接口：

* 判断是否有待取的URL，方法：has_new_url():True/False
* 添加新的URL到未爬取的集合中，方法：add_new_url(url),add_new_urls(urls)
* 获取一个未爬取的URL，方法：get_new_url():str
* 获取未爬取URL集合的大小，方法：num_of_new_urls(): int
* 获取已爬取URL集合的大小，方法：num_of_used_urls(): int

实现：

In [1]:
class UrlManager:
    '''URL管理器：负责管理URL链接'''
    
    # 构造方法
    def __init__(self):
        # 未爬取的URL集合
        self.new_urls = set()
        # 已爬取的URL集合
        self.used_urls = set()
        
    def has_new_url(self):
        '''
        判断是否有待取的URL
        :return: True/False
        '''
        return self.num_of_new_urls() != 0
    
    def add_new_url(self, url):
        '''
        添加新的URL
        :param url: 单个URL
        :return: None
        '''
        if url is None:
            return
        if url not in self.new_urls and url not in self.used_urls:
            self.new_urls.add(url)
    
    def add_new_urls(self, urls):
        '''
        添加多个新的URL
        :param url: 多个URL
        :return: None
        '''
        if urls is None or len(urls) == 0:
            return
        for url in urls:
            self.add_new_url(url)
    
    def get_new_url(self):
        '''
        获取一个未爬取的URL
        :return: str
        '''
        url = self.new_urls.pop()
        self.used_urls.add(url)
        return url
            
    def num_of_new_urls(self):
        '''
        获取未爬取URL的数量
        return: int
        '''
        return len(self.new_urls)
    
    def num_of_used_urls(self):
        '''
        获取已爬取URL的数量
        return: int
        '''
        return len(self.used_urls)
    

## 4.3 HTML下载器

功能：下载网页

接口：

* 从指定的URL下载HTML页面，方法：download(url): str（html文本）

实现：

In [2]:
import requests

class HtmlDownloader:
    '''
    HTML下载器：下载网页HTML文本
    '''
    def download(self, url):
        '''
        从指定的URL下载HTML文本内容
        :return: str
        '''
        if url is None:
            return None
        user_agent = 'Mozilla/4.0 (compatible); MSIE 5.5; Windows NT)'
        headers = {'User-Agent': user_agent}
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            r.encoding = 'utf-8'
            return r.text
        return None

## 4.4 HTML解析器

### 父类（抽象）功能接口

功能：解析网页，提取数据和链接

接口：

* 解析网页内容，抽取URL和数据，方法：parse(html_content):(urls, data)

实现：

In [3]:
class HtmlParser:
    '''
    HTML解析器（抽象类）：解析网页，提取URL和数据
    '''
    
    def parse(self, html_content):
        '''
        解析网页内容，抽取URL和数据
        :return :tuple(urls,data)
        '''
        raise NotImplementedError("必须重写该方法！")
    

### 具体类：搜狗Top500网页解析器

实现技术：

基于BeautifulSoup库的解析器，用来解析搜狗TOP500网页数据

实现：

In [4]:
from bs4 import BeautifulSoup

class SougouTop500HtmlParser(HtmlParser):
    '''
    基于BeautifulSoup库的HTML解析器，用来解析搜狗TOP500网页数据
    '''
    def parse(self, html_content):
        
        # 内嵌函数，用于提取数据
        def parse_data(html_content):
            music_div = soup.find('div', class_='pc_temp_songlist')
            music_list = music_div.find_all('li')
            musics = []
            for music_li in music_list:
                rank = music_li.find('span', class_='pc_temp_num')
                rank = rank.text.strip()
                title = music_li.get('title')
                artist = title.split('-')[0].strip()
                title = title.split('-')[1].strip()
                time = music_li.find('span', class_='pc_temp_time')
                time = time.text.strip()
                music_info = {'rank': rank, 
                              'artist': artist, 
                              'title': title, 
                              'time': time
                             }
                musics.append(music_info)
            return musics
        # 内嵌函数，用于提取URL（本站无需提取URL）
        def parse_urls(html_content):
            pass
        
        if html_content is None:
            return None
        soup = BeautifulSoup(html_content, 'html.parser')
        # 提取数据
        data = parse_data(html_content)
        # 提取URL
        urls = parse_urls(html_content)
        
        return urls, data      

## 4.5 数据存储器

包含：数据缓冲区: data_buffer

功能：存储数据到文件或者数据库中

接口：

* 存储数据集,方法：write(filename, data)

实现：

In [5]:
import csv

class DataWriter:
    '''
    数据存储器：存储数据到文件或者数据库中
    默认实现，存储为CSV格式
    '''
    def __init__(self, filename):
        self.filename = filename
        self.data_buffer = []
        
    def put(self, data):
        self.data_buffer.extend(data)
    
    def get(self):
        return self.data_buffer
        
    def save(self):
        with open(self.filename, 'a', encoding='utf-8') as f:
            fieldnames = self.data_buffer[0]
            dict_writer = csv.DictWriter(f, fieldnames)
            dict_writer.writeheader()
            dict_writer.writerows(self.data_buffer) 

## 4.6 爬虫调度器

包含：

* URL管理器对象
* HTML下载器对象
* HTML解析器对象
* 数据存储器对象

功能：协调上述四个模块，爬取指定的URL列表

接口：

* 爬取指定的URL列表：方法：crawl(start_urls)

实现：

In [6]:
class SpiderScheduler:
    '''
    爬虫调度器：协调上述四个模块，爬取指定的URL列表
    '''
    # 构造方法
    def __init__(self, html_parser = None, data_writer = None):
        self.url_manager = UrlManager()
        self.html_downloader = HtmlDownloader()
        if html_parser is None:
            self.html_parser = HtmlParser()
        else:
            self.html_parser = html_parser
        if data_writer is None:
            self.data_writer = DataWriter("spider_data.csv")
        else:
            self.data_writer = data_writer
    
    def crawl(self, start_urls):
        self.url_manager.add_new_urls(start_urls)
        while(self.url_manager.has_new_url()):
            new_url = self.url_manager.get_new_url()
            print('正在爬取：{}'.format(new_url), end='...')
            
            try:
                html_text = self.html_downloader.download(new_url)
                urls, data = self.html_parser.parse(html_text)
                if (urls is not None) and (len(urls)) > 0:
                    self.url_manager.add_new_urls(urls)
                if data is not None:
                    self.data_writer.put(data)
                num_of_used = self.url_manager.num_of_used_urls()
                num_of_new = self.url_manager.num_of_new_urls()
                print('已完成{0}，剩余{1}.'.format(num_of_used, num_of_new))                
            except Exception as e:
                print("发生错误：{}".format(e))
                raise e

        self.data_writer.save()

In [7]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [8]:
spider = SpiderScheduler(html_parser = SougouTop500HtmlParser())
spider.crawl(['http://www.kugou.com/yy/rank/home/1-8888.html'])

正在爬取：http://www.kugou.com/yy/rank/home/1-8888.html...已完成1，剩余0.


In [13]:
!rm spider_data.csv

In [10]:
from myspider import SpiderScheduler

spider = SpiderScheduler(html_parser = SougouTop500HtmlParser())
start_urls = ['http://www.kugou.com/yy/rank/home/{0}-8888.html'.format(i) for i in range(1,6)]
start_urls

['http://www.kugou.com/yy/rank/home/1-8888.html',
 'http://www.kugou.com/yy/rank/home/2-8888.html',
 'http://www.kugou.com/yy/rank/home/3-8888.html',
 'http://www.kugou.com/yy/rank/home/4-8888.html',
 'http://www.kugou.com/yy/rank/home/5-8888.html']

In [11]:
spider.crawl(start_urls)

正在爬取：http://www.kugou.com/yy/rank/home/2-8888.html...已完成1，剩余4.
正在爬取：http://www.kugou.com/yy/rank/home/4-8888.html...已完成2，剩余3.
正在爬取：http://www.kugou.com/yy/rank/home/3-8888.html...已完成3，剩余2.
正在爬取：http://www.kugou.com/yy/rank/home/1-8888.html...已完成4，剩余1.
正在爬取：http://www.kugou.com/yy/rank/home/5-8888.html...已完成5，剩余0.


In [12]:
!cat spider_data.csv